# 🚛 Resilient Routing - Senior Demo

This notebook demonstrates the core logic of the Resilient Routing Decision Support System programmatically.
It covers:
1. **Data Generation**: Creating realistic points in SP using Gaussian clusters.
2. **Optimization**: Solving the VRP with Time Windows using OR-Tools (AI) vs Greedy Baseline (Manual).
3. **Simulation**: Running the routes with dynamic weather and calculating financial impact (Saving).

In [1]:
import sys
import os
import pandas as pd
import simpy
import numpy as np
import datetime

# Add project root to path to import src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.data_gen import RealWorldVRPCreator
from src.optimization import RouteOptimizer
from src.simulation import WeatherService, LogisticsSimulator

## 1. Generate Real-World Data (São Paulo)
We generate realistic customer locations clustered around major hubs in SP, with realistic travel times (20km/h avg speed + urban friction).

In [2]:
# Configuration
NUM_CUSTOMERS = 50
SERVICE_TIME = 30 # minutes
SHIFT_DURATION = 600 # 10 hours (8am - 6pm)
NUM_VEHICLES = 50 # Allow solver to choose optimal number

creator = RealWorldVRPCreator(num_customers=NUM_CUSTOMERS)
data_model = creator.create_data_model(
    max_time_minutes=SHIFT_DURATION, 
    service_time=SERVICE_TIME, 
    num_vehicles=NUM_VEHICLES
)

print(f"Generated {len(data_model['locations'])} locations.")
print(f"Depot at: {data_model['locations'][0].lat}, {data_model['locations'][0].lon}")

Generated 51 locations.
Depot at: -23.5505, -46.6333


## 2. Optimize Routes (AI vs Manual)
We compare two approaches:
1. **AI Optimization (OR-Tools):** Minimizes global cost considering time windows and capacity.
2. **Manual Baseline (Greedy):** Simulates a human planner picking the nearest next customer.

In [3]:
optimizer = RouteOptimizer(data_model)

# 1. AI Optimization
routes_opt = optimizer.solve()
print(f"AI Optimization complete. Found {len([r for r in routes_opt if r])} active routes.")

# 2. Manual Baseline
routes_manual = optimizer.solve_greedy()
print(f"Manual Baseline complete. Found {len([r for r in routes_manual if r])} active routes.")

AI Optimization complete. Found 6 active routes.
Manual Baseline complete. Found 6 active routes.


## 3. Simulate Weather Impact (SimPy)
We run both sets of routes through the same stochastic weather simulation to see how they perform in reality.

In [4]:
env_opt = simpy.Environment()
env_manual = simpy.Environment()

# Weather Configuration
weather_service = WeatherService(storm_prob=0.1, rain_prob=0.2)

# Run AI Simulation
simulator_opt = LogisticsSimulator(env_opt, data_model, routes_opt, weather_service)
results_opt = simulator_opt.run()

# Run Manual Simulation
simulator_manual = LogisticsSimulator(env_manual, data_model, routes_manual, weather_service)
results_manual = simulator_manual.run()

print("Simulation Complete.")

Simulation Complete.


## 4. Financial Analysis & Saving
We calculate the total cost (Operational + Overtime) for both scenarios and determine the Saving generated by the AI.

In [5]:
CUSTO_OPERACIONAL_MIN = 1.00 # R$/min

# AI Costs
total_time_opt = results_opt["Tempo Real"].sum()
cost_ops_opt = total_time_opt * CUSTO_OPERACIONAL_MIN
cost_overtime_opt = results_opt["Custo Hora Extra"].sum()
total_cost_opt = cost_ops_opt + cost_overtime_opt

# Manual Costs
total_time_manual = results_manual["Tempo Real"].sum()
cost_ops_manual = total_time_manual * CUSTO_OPERACIONAL_MIN
cost_overtime_manual = results_manual["Custo Hora Extra"].sum()
total_cost_manual = cost_ops_manual + cost_overtime_manual

# Saving
saving = total_cost_manual - total_cost_opt
saving_pct = (saving / total_cost_manual * 100) if total_cost_manual > 0 else 0

print(f"--- Financial Results ---")
print(f"Total Cost (AI):     R$ {total_cost_opt:,.2f}")
print(f"Total Cost (Manual): R$ {total_cost_manual:,.2f}")
print(f"SAVING:              R$ {saving:,.2f} ({saving_pct:.1f}%)")

print("\n--- Detailed AI Results (Head) ---")
display(results_opt.head())

--- Financial Results ---
Total Cost (AI):     R$ 825.20
Total Cost (Manual): R$ 2,465.02
SAVING:              R$ 1,639.82 (66.5%)

--- Detailed AI Results (Head) ---


,Veículo,De,Para,Clima,Tempo Base,Tempo Real,Hora Chegada,Atraso,Custo Hora Extra
0,45,0,5,Ensolarado,5,5.00,5.00,0.00,0.0
1,44,0,9,Ensolarado,6,6.00,6.00,0.00,0.0
2,47,0,10,Ensolarado,7,7.00,7.00,0.00,0.0
3,49,0,20,Ensolarado,11,11.00,11.00,0.00,0.0
4,44,9,12,Chuva Leve,5,6.25,42.25,1.25,0.0
